In [2]:
%load_ext autoreload
%autoreload

In [3]:
%aimport sys
sys.path.insert(0,'..')

In [4]:
%env TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data

env: TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data


In [12]:
from os.path import dirname, realpath, join


from textmorph import data
from textmorph.text_generation.training_run import GenData
from gtd.ml.torch.utils import similar_size_batches

In [7]:
data_dir = join(data.workspace.root, "textgen_dataset")

In [10]:
source_data = GenData(data_dir)

[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/textgen_dataset/IVR_text_HiGe_SCinternal


In [11]:
source_data.data

[[u'ich',
  u'habe',
  u'eine',
  u'frage',
  u'zu',
  u'meiner',
  u'handyrechnung',
  u'der',
  u'betrag',
  u'stimmt',
  u'nicht'],
 [u'ich', u'habe', u'eine', u'frage', u'zu', u'meiner', u'rechnung'],
 [u'ich', u'habe', u'eine', u'frage', u'zu', u'meiner', u'rechnung'],
 [u'tschau',
  u'zusammen',
  u'ich',
  u'komme',
  u'bei',
  u'meiner',
  u'rechnung',
  u'nicht',
  u'draus',
  u'k\xf6nnt',
  u'ihr',
  u'mir',
  u'helfen'],
 [u'ich',
  u'habe',
  u'wieder',
  u'mal',
  u'ein',
  u'problem',
  u'mit',
  u'der',
  u'rechnung'],
 [u'ich', u'habe', u'ein', u'problem', u'meine', u'rechnung'],
 [u'ich',
  u'habe',
  u'eine',
  u'falsche',
  u'rechnung',
  u'f\xfcr',
  u'mein',
  u'internet',
  u'bekommen'],
 [u'ich', u'habe', u'ein', u'problem', u'mit', u'meiner', u'rechnung'],
 [u'ich', u'kann', u'meine', u'rechnung', u'nicht', u'bezahlen'],
 [u'salut',
  u'zusammen',
  u'ich',
  u'habe',
  u'ein',
  u'problem',
  u'mit',
  u'meiner',
  u'rechnung'],
 [u'ich', u'habe', u'ein', u'rec

In [15]:
similar_size_batches(source_data.data, 32, size=lambda x: len(x))

Optimized batches: reduced cost from 7904 (naive) to 5248 (0.336032388664% reduction).
Optimal (batch_size=1) would be 2061.


[[[u'salut',
   u'zusammen',
   u'ich',
   u'kann',
   u'meine',
   u'sim',
   u'karte',
   u'nicht',
   u'aktivieren',
   u'k\xf6nnt',
   u'ihr',
   u'mir',
   u'helfen'],
  [u'hallo',
   u'ich',
   u'habe',
   u'ein',
   u'problem',
   u'mit',
   u'meiner',
   u'rechnung',
   u'ich',
   u'verstehe',
   u'die',
   u'zahlen',
   u'nicht'],
  [u'ich',
   u'komme',
   u'\xfcberhaupt',
   u'nicht',
   u'raus',
   u'bei',
   u'meiner',
   u'rechnung.',
   u'k\xf6nnt',
   u'ihr',
   u'mir',
   u'bitte',
   u'helfen?'],
  [u'salut',
   u'zusammen',
   u'hier',
   u'ist',
   u'micha',
   u'baeriswyl',
   u'ich',
   u'habe',
   u'ein',
   u'problem',
   u'mit',
   u'meiner',
   u'rechnung'],
  [u'hallo',
   u'ich',
   u'bin',
   u'der',
   u'marcel',
   u'und',
   u'ich',
   u'habe',
   u'ein',
   u'problem',
   u'mit',
   u'meiner',
   u'rechnung.'],
  [u'ja',
   u'hallo',
   u'hier',
   u'ist',
   u'der',
   u'julian.',
   u'ich',
   u'habe',
   u'ein',
   u'problem',
   u'mit',
   u'meiner'

In [14]:
type(source_data)

textmorph.text_generation.training_run.GenData